In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [3]:
fake=pd.read_csv('/content/drive/MyDrive/ramu data/Fake.csv')
real=pd.read_csv('/content/drive/MyDrive/ramu data/True.csv')


In [4]:
real=real.drop(8970, axis=0)

In [5]:
unknown_publishers=[]
for index,row in enumerate(real.text.values):
  try:
    record=row.split('-',maxsplit=1)
    record[1]
    assert(len(record[0])<120)
  except:
    unknown_publishers.append(index)

In [7]:
publisher=[]
tmp_text=[]
for index,row in enumerate(real.text.values):
  if index in unknown_publishers:
    tmp_text.append(row)
    publisher.append('Unknown')
  else:
    record=row.split('-',maxsplit=1)
    publisher.append(record[0].strip())
    tmp_text.append(record[1].strip())

In [8]:
real['publisher']=publisher
real['text']=tmp_text

In [11]:
real['text']=real['title'] + " " + real['text']
fake['text']=fake['title'] + " " + fake['text']
real['class']=1
fake['class']=0

In [12]:
real=real[['text','class']]
fake=fake[['text','class']]

In [14]:
data=pd.concat((real,fake),axis=0)

In [17]:
data['text']=data['text'].apply(lambda x: str(x).lower())

In [20]:
data['text']=data['text'].apply(lambda x: re.sub(r"[^\w\s]","",str(x)))
data['text']=data['text'].apply(lambda x: re.sub(r"\d","",str(x)))

In [22]:
import gensim

In [23]:
y=data['class'].values
X = [d.split() for d in data['text'].tolist()]

In [30]:
print(X[0])

['as', 'us', 'budget', 'fight', 'looms', 'republicans', 'flip', 'their', 'fiscal', 'script', 'the', 'head', 'of', 'a', 'conservative', 'republican', 'faction', 'in', 'the', 'us', 'congress', 'who', 'voted', 'this', 'month', 'for', 'a', 'huge', 'expansion', 'of', 'the', 'national', 'debt', 'to', 'pay', 'for', 'tax', 'cuts', 'called', 'himself', 'a', 'fiscal', 'conservative', 'on', 'sunday', 'and', 'urged', 'budget', 'restraint', 'in', 'in', 'keeping', 'with', 'a', 'sharp', 'pivot', 'under', 'way', 'among', 'republicans', 'us', 'representative', 'mark', 'meadows', 'speaking', 'on', 'cbs', 'face', 'the', 'nation', 'drew', 'a', 'hard', 'line', 'on', 'federal', 'spending', 'which', 'lawmakers', 'are', 'bracing', 'to', 'do', 'battle', 'over', 'in', 'january', 'when', 'they', 'return', 'from', 'the', 'holidays', 'on', 'wednesday', 'lawmakers', 'will', 'begin', 'trying', 'to', 'pass', 'a', 'federal', 'budget', 'in', 'a', 'fight', 'likely', 'to', 'be', 'linked', 'to', 'other', 'issues', 'such',

In [34]:
DIM=100
w2v_model=gensim.models.Word2Vec(sentences=X,vector_size=DIM,window=10,min_count=1)

In [36]:
len(w2v_model.wv.key_to_index)

215452

In [42]:
w2v_model.wv.most_similar('us')

[('american', 0.5684501528739929),
 ('washingtons', 0.49351492524147034),
 ('rotor', 0.48089921474456787),
 ('iranian', 0.48032745718955994),
 ('iran', 0.46686387062072754),
 ('nato', 0.4492739737033844),
 ('irans', 0.4471930265426636),
 ('turkey', 0.44272613525390625),
 ('russian', 0.43996119499206543),
 ('saudi', 0.43163198232650757)]

In [43]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(X)
X=tokenizer.texts_to_sequences(X)

In [46]:
maxlen=1000
X=pad_sequences(X,maxlen=maxlen)
vocab_size=len(tokenizer.word_index)+1
vocab=tokenizer.word_index

In [48]:
def get_weight_matrix(model):
  weight_matrix=np.zeros((vocab_size,DIM))
  for word, i in vocab.items():
    weight_matrix[i]=model.wv[word]
  return weight_matrix

In [49]:
embedding_vectors=get_weight_matrix(w2v_model)

In [51]:
model=Sequential()
model.add(Embedding(vocab_size,output_dim=DIM, weights=[embedding_vectors],input_length=maxlen,trainable=False))
model.add(LSTM(units=128))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])

In [52]:
X_train,X_test,y_train,y_test=train_test_split(X,y)

In [53]:
model.fit(X_train,y_train,validation_split=0.3,epochs=6)

Epoch 1/6
737/737 [==============================] - 35s 38ms/step - loss: 0.1413 - acc: 0.9510 - val_loss: 0.0805 - val_acc: 0.9746
Epoch 2/6
737/737 [==============================] - 27s 36ms/step - loss: 0.0749 - acc: 0.9745 - val_loss: 0.0497 - val_acc: 0.9836
Epoch 3/6
737/737 [==============================] - 28s 38ms/step - loss: 0.0278 - acc: 0.9910 - val_loss: 0.0420 - val_acc: 0.9862
Epoch 4/6
737/737 [==============================] - 27s 37ms/step - loss: 0.0286 - acc: 0.9907 - val_loss: 0.0662 - val_acc: 0.9753
Epoch 5/6
737/737 [==============================] - 27s 36ms/step - loss: 0.0140 - acc: 0.9953 - val_loss: 0.0269 - val_acc: 0.9916
Epoch 6/6
737/737 [==============================] - 27s 37ms/step - loss: 0.0119 - acc: 0.9966 - val_loss: 0.0227 - val_acc: 0.9934


In [55]:
import pickle
pickle.dump(model,open('model.pkl','wb'))